In [ ]:
from depmapomics.tracker import getCCLETracker

tracker = getCCLETracker()

In [ ]:
dropped_from_rna_legacy_v1_to_v3 = {'ACH-000047','ACH-000309','ACH-000426','ACH-000658','ACH-000727','ACH-000795','ACH-000979','ACH-000992'}
tracker[tracker['arxspan_id'].isin(dropped_from_rna_legacy_v1_to_v3) & (tracker['datatype']=='rna')][['arxspan_id', 'low_quality', 'blacklist']]

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
CCLE_expression = tc.get(name='tentative-virtual-d84e', version=33, file='CCLE_expression')


In [ ]:
CCLE_gene_cn = tc.get(name='tentative-virtual-d84e', version=33, file='CCLE_gene_cn')


In [ ]:
'ACH-000658' in set(CCLE_expression.index)

In [ ]:
'ACH-000658' in set(CCLE_gene_cn.index)

In [ ]:
dropped_from_rna_legacy_v1_to_v3 & set(CCLE_expression.index)

In [ ]:
SAMPLESETNAME = '21Q3'

In [ ]:
import pandas as pd
transcripts_tpm = pd.read_csv('temp/' + SAMPLESETNAME + '/transcripts_tpm_logp1.csv', index_col=0)

In [ ]:
genes_tpm = pd.read_csv('temp/'+SAMPLESETNAME+'/genes_tpm_logp1.csv',index_col=0)

In [ ]:
genes_expected_count = pd.read_csv('temp/' + SAMPLESETNAME + '/genes_expected_count.csv', index_col=0)

In [ ]:
proteincoding_genes_expected_count = pd.read_csv('temp/' + SAMPLESETNAME + '/proteincoding_genes_expected_count.csv', index_col=0)

In [ ]:




proteincoding_genes_tpm = pd.read_csv('temp/' + SAMPLESETNAME + '/proteincoding_genes_tpm_logp1.csv', index_col=0)
transcripts_expected_count = pd.read_csv('temp/' + SAMPLESETNAME + '/transcripts_expected_count.csv', index_col=0)
#enrichments = pd.read_csv('temp/' + SAMPLESETNAME + '/gene_sets_all.csv', index_col=0)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

# CCLE_mutations_21q2 = tc.get(name='internal-21q2-9d16', version=17, file='CCLE_mutations')
# CCLE_expression_21q2 = tc.get(name='internal-21q2-9d16', version=17, file='CCLE_expression')
# CCLE_expression_21q3 = tc.get(name='tentative-virtual-d84e', version=30, file='CCLE_expression')
# CCLE_mutations_21q3 = tc.get(name='tentative-virtual-d84e', version=30, file='CCLE_mutations')

legacy_rna_1 = tc.get(name='mutations-da6a', version=1, file='legacy_rna_somatic_mutations')
legacy_rna_3 = tc.get(name='mutations-da6a', file='legacy_rna', version=3)

# legacy_wes_sanger_v3 = tc.get(name='mutations-da6a', version=3, file='legacy_wes_sanger')
# legacy_wes_sanger_v1 = tc.get(name='mutations-da6a', version=1, file='legacy_wes_sanger_somatic_mutations')


In [ ]:
import pandas as pd
def cleanup_legacy(legacy_maf, ac_col):
  print('cleaning up mutation legacy data for column {}'.format(ac_col))
  legacy_maf = legacy_maf.drop(['Unnamed: 0', 'Tumor_Sample_Barcode'], axis=1).rename(
      columns={'Tumor_Seq_Allele1': 'Tumor_Allele'})
  freqs = legacy_maf.apply(lambda x: [int(y) for y in x[ac_col].split(':')], result_type='expand', axis=1)
  freqs.columns = ['t_alt_count', 't_ref_count']
  freqs['tumor_f'] = freqs.astype(float).apply(
      lambda row: row['t_alt_count'] / (row['t_alt_count'] + row['t_ref_count']), axis=1)
  legacy_maf = pd.concat([legacy_maf, freqs], axis=1)
  return legacy_maf


In [ ]:
legacy_rna = cleanup_legacy(legacy_rna_1, 'RNAseq_AC')

In [ ]:
set(legacy_rna.columns) - set(legacy_rna_3.columns)

In [ ]:
set(legacy_rna_3.columns) - set(legacy_rna.columns)

In [ ]:
legacy_rna

In [ ]:
set(legacy_rna.columns) - set(legacy_rna_3.columns)

In [ ]:
mergecols = ['DepMap_ID', 'Chromosome', 'Start_position', 'End_position', 'Tumor_Allele']

def test_row_counts_match(legacy_wgs_exoniconly_v1, legacy_wgs_exoniconly_v3):
  counts = pd.concat([legacy_wgs_exoniconly_v1.groupby('DepMap_ID').size(), 
            legacy_wgs_exoniconly_v3.groupby('DepMap_ID').size()], 
            join = 'inner', keys=['v1', 'v3'], axis=1)
  assert (counts['v1'] == counts['v3']).all(), counts[counts['v1'] != counts['v3']]  

  assert not legacy_wgs_exoniconly_v1[mergecols].duplicated().any()
  assert not legacy_wgs_exoniconly_v3[mergecols].duplicated().any()

def test_shared_mutations(legacy_wgs_exoniconly_v1, legacy_wgs_exoniconly_v3, test=True):
  shared_lines = set(legacy_wgs_exoniconly_v1['DepMap_ID']) & set(legacy_wgs_exoniconly_v3['DepMap_ID'])

  merged = pd.merge(legacy_wgs_exoniconly_v1[legacy_wgs_exoniconly_v1['DepMap_ID'].isin(shared_lines)],
                    legacy_wgs_exoniconly_v3[legacy_wgs_exoniconly_v3['DepMap_ID'].isin(shared_lines)],
                    on=mergecols, indicator=True, how='outer')
  if test:
    assert (merged['_merge'] == 'both').all(), 'some mutations are not shared between the two datasets'
  return merged
  

import numpy as np

def test_fraction_of_unequal_columns_from_merged_file(dataframes_merged, expected_changed_cols):
    dataframes_merged.drop([x+'_x' for x in expected_changed_cols], inplace=True, axis=1)
    dataframes_merged.drop([x+'_y' for x in expected_changed_cols], inplace=True, axis=1)
    cols = list(set([x[:-2] for x in dataframes_merged.columns if x.endswith('_x') | x.endswith('_y')]))
    dataframe_merge_both = dataframes_merged[dataframes_merged['_merge'] == 'both']
    dataframe_merge_both.set_index('DepMap_ID', inplace=True)
    unequal_values = pd.DataFrame(index=dataframe_merge_both.index, columns=cols)
    cols_dtype = dataframe_merge_both[[col+'_x' for col in cols]].dtypes
    equal_nonNA = lambda a, b: (a == b) | ((a != a) & (b != b)) # this is a regular equality tests with the exception that NA==NA
    almost_equal_nonNA = lambda a, b: np.isclose(a, b) | ((a != a) & (b != b)) # this is a regular equality tests with the exception that NA==NA
    for col in cols:
        if cols_dtype[col+'_x'] == np.dtype('float64'): # otherwise very close values will look different
            unequal_values[col] = ~almost_equal_nonNA(dataframe_merge_both[col+'_x'], dataframe_merge_both[col+'_y'])
        else:
            unequal_values[col] = ~equal_nonNA(dataframe_merge_both[col+'_x'], dataframe_merge_both[col+'_y'])

    unequal_columns = unequal_values.agg(['mean', 'sum']).T
    unequal_columns.sort_values('mean', ascending=False, inplace=True)
    unequal_columns['sum'] = unequal_columns['sum'].astype(int)
    unequal_columns = unequal_columns[unequal_columns['sum'] > 0]
    unequal_columns.rename(columns={'mean': 'freq', 'sum': 'count'}, inplace=True)

    unequal_values_sum = unequal_values.groupby('DepMap_ID').sum()
    unequal_values_sum = unequal_values_sum[(unequal_values_sum > 0).any(axis=1)]
    unequal_values_sum = unequal_values_sum.loc[:, (unequal_values_sum > 0).all()]

    return unequal_values, unequal_columns, unequal_values_sum
    # assert unequal_columns.empty

# unequal_values, unequal_columns, unequal_values_sum = test_fraction_of_unequal_columns_from_merged_file(merged, [])

In [ ]:
merged = test_shared_mutations(legacy_rna, legacy_rna_3, test=False)

In [ ]:
counts = pd.concat([legacy_rna.groupby('DepMap_ID').size(), 
          legacy_rna_3.groupby('DepMap_ID').size()], 
          join = 'inner', keys=['v1', 'v3'], axis=1)


In [ ]:
merged[(merged['DepMap_ID'] == 'ACH-000192') & (merged['_merge']!='both')]

In [ ]:
counts[counts['v1'] != counts['v3']]

In [ ]:
unequal_values, unequal_columns, unequal_values_sum = test_fraction_of_unequal_columns_from_merged_file(merged, [])

In [ ]:
unequal_columns

In [ ]:
merged[merged['Genome_Change_x'] != merged['Genome_Change_y']]

In [ ]:
test_row_counts_match(legacy_rna, legacy_rna_3)

In [ ]:

legacy_rna_3

In [ ]:
arxspan = 'ACH-000426'

print(arxspan in set(CCLE_expression_21q2.index), arxspan in set(CCLE_expression_21q3.index))
print(CCLE_mutations_21q2[CCLE_mutations_21q2['DepMap_ID'] == arxspan]['RNAseq_AC'].notnull().any(),\
CCLE_mutations_21q3[CCLE_mutations_21q3['DepMap_ID'] == arxspan]['RNAseq_AC'].notnull().any()  )
print(arxspan in set(legacy_rna_1['DepMap_ID']), arxspan in set(legacy_rna_3['DepMap_ID']))

In [ ]:
set(legacy_rna_3['DepMap_ID']) - set(legacy_rna_1['DepMap_ID'])

In [ ]:
arxspans = '''
E         ACH-000195         11    11    11        11           10     00
E         ACH-001543         11    00    00        00           01     00
E         ACH-000090         10    11    11        11           00     11
E         ACH-000992         11    11    00        10           00     00
E         ACH-000641         11    11    11        11           10     00
E         ACH-001212         11    00    00        10           00     00
E         ACH-000727         11    11    11        10           11     11
E         ACH-000658         10    11    11        10           00     00
E         ACH-000426         11    11    11        10           00     00
E         ACH-000901         10    11    00        11           00     00
E         ACH-000979         11    10    11        10           11     11
E         ACH-001709         11    00    00        00           10     00
E         ACH-000795         11    11    11        10           11     00
E         ACH-000309         11    11    11        10           11     11
E         ACH-000047         11    11    11        10           11     11
E         ACH-000961         10    11    11        11           00     11
'''
arxspans = {x.strip().split()[0] for x in arxspans.split('E') if x != '\n'}

In [ ]:
arxspans - set(legacy_rna_1['DepMap_ID'])

In [ ]:
arxspans - set(legacy_rna_3['DepMap_ID'])

In [ ]:
set(legacy_rna_1['DepMap_ID']) - set(legacy_rna_3['DepMap_ID'])

In [ ]:
set(legacy_wes_sanger_v3['DepMap_ID']) - set(legacy_wes_sanger_v1['DepMap_ID'])

In [ ]:
set(legacy_wes_sanger_v1['DepMap_ID']) - set(legacy_wes_sanger_v3['DepMap_ID'])

In [ ]:
legacy_rna_1

In [ ]:
legacy_wes_sanger_v1

In [ ]:
CCLE_mutations.columns

In [ ]:
CCLE_mutations[['Reference_Allele', 'Tumor_Seq_Allele1']]


In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

CCLE_fusions = tc.get(name='tentative-virtual-d84e', version=30, file='CCLE_fusions')
CCLE_fusions_unfiltered = tc.get(name='tentative-virtual-d84e', version=30, file='CCLE_fusions_unfiltered')


In [ ]:
len(CCLE_fusions[CCLE_fusions['DepMap_ID'] == 'ACH-000756'])

In [ ]:
len(CCLE_fusions_unfiltered[CCLE_fusions_unfiltered['DepMap_ID'] == 'ACH-000756'])

In [ ]:
CCLE_fusions = tc.get(name='internal-21q2-9d16', version=17, file='CCLE_fusions')
CCLE_fusions_unfiltered = tc.get(name='internal-21q2-9d16', version=17, file='CCLE_fusions_unfiltered')


In [ ]:
len(CCLE_fusions[CCLE_fusions['DepMap_ID'] == 'ACH-000756'])

In [ ]:
len(CCLE_fusions_unfiltered[CCLE_fusions_unfiltered['DepMap_ID'] == 'ACH-000756'])

In [ ]:
arxspans = '''
ACH-000756
ACH-000833
ACH-001096
ACH-001163
ACH-001164
ACH-001196
'''

arxspans = set(arxspans.split())

In [ ]:
from genepy.utils import helper as h
renaming = h.fileToDict("temp/21Q3/rna_sample_renaming.json")

In [ ]:
renaming.pop('CDS-tTwJLo')
renaming.pop('CDS-Fauu92')
renaming.update({'CDS-pvNwpR': 'ACH-000833', 'CDS-V5yMiU': 'ACH-001164'})

In [ ]:
{k:v for k,v in renaming.items() if v in arxspans}

In [ ]:
from depmapomics.tracker import getCCLETracker
tracker = getCCLETracker()

In [ ]:
tracker_subset = tracker[(tracker['datatype']=='rna') & tracker['arxspan_id'].isin(arxspans)]
tracker_subset.sort_values(by=['arxspan_id', 'version'])

In [ ]:
import pandas as pd

fusions_all = pd.read_csv("temp/21Q3/fusions_all.csv")
fusions_latest = pd.read_csv("temp/21Q3/fusions_latest.csv")

from taigapy import TaigaClient
tc = TaigaClient()
CCLE_fusions = tc.get(name='internal-21q2-9d16', version=17, file='CCLE_fusions')

In [ ]:
cds_id = 'CDS-tTwJLo'	
arxspan_id = 'ACH-000833'
x1 = fusions_all.loc[fusions_all['DepMap_ID'] == cds_id, 'JunctionReadCount']
x2 = fusions_latest.loc[fusions_latest['DepMap_ID'] == arxspan_id, 'JunctionReadCount']
x3 = CCLE_fusions.loc[fusions_latest['DepMap_ID'] == arxspan_id, 'JunctionReadCount']
assert x1.reset_index(drop=True).equals(x2.reset_index(drop=True))

In [ ]:
import dalmatian as dm

wm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq')

In [ ]:
sample_sets = wm.get_sample_sets()

In [ ]:
arxspans ='''
ACH-000756
ACH-000833
ACH-001096
ACH-001163
ACH-001164
ACH-001196
'''
arxspans = arxspans.split()

from depmapomics.tracker import getCCLETracker
tracker = getCCLETracker()


In [ ]:
tracker_subset = tracker[tracker['arxspan_id'].isin(arxspans) & (tracker['datatype'] == 'rna')]

In [ ]:
sample_sets['samples'].map(lambda x: set(x) & set(tracker_subset.index))

In [ ]:
set(tracker_subset.index) - set(sample_sets.loc['star_fusion_workflow_2020-10-15T18-40-03']['samples']) 

In [ ]:
  FUSION_COLNAME = ['DepMap_ID', 'FusionName', 'JunctionReadCount',
                  'SpanningFragCount', 'SpliceType', 'LeftGene', 'LeftBreakpoint',
                  'RightGene', 'RightBreakpoint', 'LargeAnchorSupport', 'FFPM',
                  'LeftBreakDinuc', 'LeftBreakEntropy', 'RightBreakDinuc',
                  'RightBreakEntropy', 'annots']
  aggregated = wm.get_sample_sets().loc['all']['fusions_star']
  fusions = pd.read_csv(aggregated, names=FUSION_COLNAME, skiprows=1, sep='\t')


In [ ]:
{'CDS-R22IHj', 'CDS-xMnTwN'} & set(tracker_subset.index)

In [ ]:
fusions[fusions['DepMap_ID'].str.startswith(cds_id)]

In [ ]:
CCLE_fusions[CCLE_fusions['DepMap_ID'].isin([arxspan_id])]

In [ ]:
x1.reset_index(drop=True).equals(x3.reset_index(drop=True))

In [ ]:
len(x1) == len(x3)

In [ ]:
fusions_all_21q2 = tc.get(name='fusions-95c9', version=7, file='fusions_all')
fusions_all_21q3 = fusions_all


In [ ]:
x21q2 = fusions_all_21q2.loc[fusions_all_21q2['DepMap_ID'] == cds_id, 'JunctionReadCount']
x21q3 = fusions_all_21q3.loc[fusions_all_21q3['DepMap_ID'] == cds_id, 'JunctionReadCount']

In [ ]:
fusions_all_21q2.loc[fusions_all_21q2['DepMap_ID'] == cds_id]

In [ ]:
merge_columns = ['DepMap_ID', 'LeftGene', 'RightGene', 'LeftBreakpoint', 'RightBreakpoint']
dataframes_merged = pd.merge(fusions_all_21q2, fusions_all_21q3, 
                       on=merge_columns, indicator=True, how='outer')

In [ ]:
dataframe_merge_both = dataframes_merged[dataframes_merged['_merge'] == 'both']

In [ ]:
dataframe_merge_both[dataframe_merge_both['JunctionReadCount_x'] != dataframe_merge_both['JunctionReadCount_y']]

In [ ]:
data_mismatch = dataframes_merged[dataframes_merged['_merge'] != 'both'][['DepMap_ID', '_merge']].drop_duplicates()
data_mismatch

In [ ]:
cds_id in set(data_mismatch['DepMap_ID'])

In [ ]:
dataframes_merged[dataframes_merged['DepMap_ID'] == cds_id]['_merge'].unique()

In [ ]:
dataframe_merge_both

In [ ]:
fusions_all_21q3.loc[fusions_all_21q3['DepMap_ID'] == cds_id]

In [ ]:
assert x21q2.reset_index(drop=True).equals(x21q3.reset_index(drop=True))

In [ ]:
x21q3

In [ ]:
    import pickle
    (files, gene_rename, trancriptLevelCols) = pickle.load(open('transcript.pkl', 'rb'))


In [ ]:
def subsetGenes(files, gene_rename, filenames=None,
                drop=[], index="transcript_id"):
  """
  # TODO: to document
  """
  print('subsetting '+index+' columns')
  rename_transcript = {}
  missing = []
  for val in filenames:
    if len(rename_transcript) == 0 and index == "transcript_id":
      for _, v in files[val].iterrows():
        if v['gene_id'].split('.')[0] in gene_rename:
          rename_transcript[v['transcript_id']] = gene_rename[
              v['gene_id'].split('.')[0]].split(' (')[0] + ' (' + v.transcript_id.split('.')[0] + ')'
        else:
          missing.append(v.gene_id.split('.')[0])
      print('missing: '+str(len(missing))+' genes')
    file = files[val].drop(columns=drop).set_index(index)
    file = file[(file.sum(1) != 0) & (file.var(1) != 0)]
    r = [i.split('.')[0] for i in file.index]
    dup = h.dups(r)
    if len(dup) > 0:
      print(dup)
      raise ValueError('duplicate '+index)
    file.index = r

    file = file.rename(index=rename_transcript if len(
        rename_transcript) != 0 else gene_rename).T
    files[val] = file
  return files



In [ ]:
for _, v in files[val].iterrows():
  if v['gene_id'].split('.')[0] in gene_rename:
    rename_transcript[v['transcript_id']] = gene_rename[
        v['gene_id'].split('.')[0]].split(' (')[0] + ' (' + v.transcript_id.split('.')[0] + ')'
  else:
    missing.append(v.gene_id.split('.')[0])


In [ ]:
file = files[val].copy()

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(gene_rename, orient='index', columns=['gene_ensg'])

In [ ]:
file['transcript_id'] = file['transcript_id'].map(lambda x: x.split('.')[0])
file['gene_id'] = file['gene_id'].map(lambda x: x.split('.')[0])

In [ ]:
merge = pd.merge(file, pd.DataFrame.from_dict(gene_rename, orient='index', columns=['gene_ensg']), 
         left_on='gene_id', right_index=True, how='left')#.set_index('gene_ensg').drop(['transcript_id',	'gene_id'], axis=1)

In [ ]:
missing2 = set(merge.loc[merge['gene_ensg'].isnull(), 'gene_id'])

In [ ]:
assert set(missing) == missing2

In [ ]:
import re
merge.index = merge.apply(lambda x: re.sub(r'ENSG\d+', x['transcript_id'], x['gene_ensg']), axis=1)

In [ ]:
# files = subsetGenes(files, gene_rename, filenames=trancriptLevelCols, drop="gene_id", index="transcript_id")

In [ ]:
files.keys()

In [ ]:
drop= "gene_id"
filenames=trancriptLevelCols
index="transcript_id"
print('subsetting '+index+' columns')
rename_transcript = {}
missing = []
val = 'transcripts_tpm'
if len(rename_transcript) == 0 and index == "transcript_id":
  for _, v in files[val].iterrows():
    if v['gene_id'].split('.')[0] in gene_rename:
      rename_transcript[v['transcript_id']] = gene_rename[
          v['gene_id'].split('.')[0]].split(' (')[0] + ' (' + v.transcript_id.split('.')[0] + ')'
    else:
      missing.append(v.gene_id.split('.')[0])
  print('missing: '+str(len(missing))+' genes')
file = files[val].drop(columns=drop).set_index(index)
file = file[(file.sum(1) != 0) & (file.var(1) != 0)]
r = [i.split('.')[0] for i in file.index]
assert len(r) == len(set(r))
# dup = h.dups(r)
# if len(dup) > 0:
#   print(dup)
#   raise ValueError('duplicate '+index)

In [ ]:
files[val]

In [ ]:
file.index = r

In [ ]:
file.rename(index={k.split('.')[0]:v for k,v in rename_transcript.items()})#.T

In [ ]:
file